In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import math
import pandas as pd
# import ikpy.chain
# import ikpy.utils.plot as plot_utils
import itertools
import csv
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import tqdm
# from chamferdist import ChamferDistance  # https://github.com/krrish94/chamferdist?tab=readme-ov-file
# import ChamferDistancePytorch.chamfer3D.dist_chamfer_3D
import kinematics
import utils
import network

In [ ]:
N_USERS = 76
M_POSES = 4096  # Number of points for each point cloud
K_TASK_POINTS = 256   # Number of points to generate in task-space
LATENT_DIM = 128
LEARNING_RATE = 1e-5
EPOCHS = 1000
BATCH_SIZE = 128
BETA_KL = 0.001 # Weight for KL loss
TRAIN_SIZE   = math.floor(0.8 * N_USERS)
VAL_SIZE     = N_USERS - TRAIN_SIZE

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

raw_data = np.loadtxt("task_sp_points.csv", delimiter=',', skiprows=1)
raw_data = torch.tensor(raw_data[:, 1:], dtype=torch.float32)
raw_data = raw_data.view(N_USERS, -1, 3)  # [N_USERS, K_TASK_POINTS, 3]
raw_data.shape

train_dataset = raw_data[0:TRAIN_SIZE].to(device)
val_dataset   = raw_data[TRAIN_SIZE:N_USERS].to(device)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = network.fROM_VAE_task(input_dim=3, latent_dim=128, num_task_points=256).to(device)
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
chamfer_loss = utils.chamfer_loss

In [ ]:
# get one batch of data and see shape
data_batch = train_dataset[0:BATCH_SIZE]
data_batch.shape  # [BATCH_SIZE, K_TASK_POINTS, 3]

In [ ]:
EPOCHS = 3000
LEARNING_RATE = 1e-5
KL_WEIGHT = 0.0001

for epoch in range(EPOCHS):
    model.train()
    total_loss_epoch = 0
    chamfer_loss_epoch = 0
    kl_loss_epoch = 0
    
    for pc in tqdm.tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} - Training", unit="batch", leave=False):
        pc = pc.to(device) # [B, M, 3]
        optimizer.zero_grad()
        
        pc_recon, mu, logvar = model(pc)
        
        loss_chamfer = chamfer_loss(pc, pc_recon)
        loss_kld = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
        loss = loss_chamfer + KL_WEIGHT * loss_kld
        
        loss.backward()
        optimizer.step()
        
        total_loss_epoch += loss.item()
        chamfer_loss_epoch += loss_chamfer.item()
        kl_loss_epoch += loss_kld.item()
        
    if (epoch + 1) % 300 == 0:
        avg_train = total_loss_epoch / len(train_loader)
        avg_chamfer = chamfer_loss_epoch / len(train_loader)
        avg_kl = kl_loss_epoch / len(train_loader)
        print(f"Epoch [{epoch+1}/{EPOCHS}] Train Loss: {avg_train:.4f}, Chamfer Loss: {avg_chamfer:.4f}, KL Loss: {avg_kl:.4f}")

In [ ]:

model.eval()
errors = 0.0

with torch.no_grad():
    for pc in tqdm.tqdm(val_loader, desc="Validation", unit="batch"):
        pc = pc.to(device)  # [B, M, 3]
        
        pc_recon, mu, logvar = model(pc)
        
        loss_chamfer = chamfer_loss(pc, pc_recon)
        loss_kld = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
        loss = loss_chamfer + KL_WEIGHT * loss_kld
        
        errors += loss.item() * pc.size(0)
avg_error = errors / len(val_loader.dataset)
print(f"Validation Loss: {avg_error:.4f}")

In [ ]:
torch.save(model.state_dict(), 'PC_VAE_76_4096.pth')

In [ ]:
# visualize
with torch.no_grad():
    x = train_dataset[5, :, :].to(device) # [B, 4]
    
    recon_x, _, _ = model(x.view(-1, K_TASK_POINTS, 3))
    
    fig = plt.figure(figsize=(10, 5))
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.scatter(x[:, 0].cpu(), x[:, 1].cpu(), x[:, 2].cpu(), c='b', s=1)
    ax1.set_title('Original Point Cloud')
    ax2 = fig.add_subplot(122, projection='3d')
    ax2.scatter(recon_x[0, :, 0].cpu(), recon_x[0, :, 1].cpu(), recon_x[0, :, 2].cpu(), c='r', s=1)
    ax2.set_title('Reconstructed Point Cloud')
    plt.show()

In [ ]:
# sanity check, interpolate
with torch.no_grad():
    pc1 = train_dataset[7:8, :, :]  # [1, K_TASK_POINTS, 3]
    pc2 = train_dataset[4:5, :, :]  # [1, K_TASK_POINTS, 3]

    z1, _ = model.encoder(pc1)
    z2, _ = model.encoder(pc2)

    z_interp = torch.linspace(0, 1, steps=10).unsqueeze(1).to(device) * z2 + (1 - torch.linspace(0, 1, steps=10).unsqueeze(1).to(device)) * z1
    pc_interp = model.decoder(z_interp)  # [10, K_TASK_POINTS, 3]
    
    # pc1_prime = model.decoder(z1)
    # pc2_prime = model.decoder(z2)
    # pc_interp = torch.cat([pc1_prime, pc2_prime], dim=0)

    pc_interp = pc_interp.cpu().numpy()
    fig = plt.figure(figsize=(25, 5))
    for i in range(pc_interp.shape[0]):
        ax = fig.add_subplot(1, pc_interp.shape[0], i+1, projection='3d')
        ax.scatter(pc_interp[i, :, 0], pc_interp[i, :, 1], pc_interp[i, :, 2], s=1)
        ax.set_title(f"Interp {i+1}")
    plt.show()

In [ ]:
# visualize dataset
fig = plt.figure(figsize=(20, 20))
for i in range(25):
    ax = fig.add_subplot(5, 5, i+1, projection='3d')
    ax.scatter(train_dataset[i:i+1, :, 0].cpu(), train_dataset[i:i+1, :, 1].cpu(), train_dataset[i:i+1, :, 2].cpu(), s=1)
    ax.set_title(f"Sample {i+1}")
plt.show()

In [ ]:


fig = plt.figure(figsize=(20, 20))
for i in range(25):
    ax = fig.add_subplot(5, 5, i+1,)
    ax.scatter(train_dataset[i:i+1, :, 0].cpu(), train_dataset[i:i+1, :, 1].cpu(), s=1)
    ax.set_title(f"Sample {i+1}")
plt.show()
    

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Make sure raw_data is a numpy array on the CPU
# If raw_data is still a torch tensor, convert it:
# data_to_plot = raw_data.cpu().numpy()
data_to_plot = raw_data.numpy() # If it's a CPU tensor

# --- Plot a Single User's Point Cloud ---
pc = data_to_plot[0]  # Get the first user's data [256, 3]

# Extract x, y, z coordinates
x = pc[:, 0]
y = pc[:, 1]
z = pc[:, 2]

# Create the 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(x, y, z, marker='.', s=10) # 's' is the marker size

ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
ax.set_zlabel('Z Coordinate')
ax.set_title('3D Scatter Plot of Task-Space Points (User 0)')
plt.show()

# --- (Optional) Plot Multiple Users ---
# This shows if all users share the same surface
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

colors = ['r', 'g', 'b', 'c', 'm', 'y']
for i in range(min(len(data_to_plot), 6)): # Plot up to 6 users
    pc = data_to_plot[i]
    ax.scatter(pc[:, 0], pc[:, 1], pc[:, 2], marker='.', s=10, c=colors[i], label=f'User {i}')

ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
ax.set_zlabel('Z Coordinate')
ax.set_title('Overlay of Multiple User Point Clouds')
ax.legend()
plt.show()

In [ ]:
# source_cloud = raw_data[0:5].to(device) 
# target_cloud = raw_data[0:5].to(device)

# loss = utils.chamfer_loss(source_cloud, target_cloud)

# print(f"Chamfer Loss (Pure PyTorch): {loss.item()}")

In [ ]:
# print(f"Python Version: {os.sys.version}")
# print(f"PyTorch Version: {torch.__version__}")
# print(f"CUDA Version (PyTorch was built with): {torch.version.cuda}")
# print(f"Is CUDA available? {torch.cuda.is_available()}")